In [1]:
pip install accelerate transformers SentencePiece  bitsandbytes einops xformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.4 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model = "NousResearch/Nous-Hermes-13b"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
    )

In [3]:
import pandas as pd

df = pd.read_json('dummy.json')

df

,n.documentName,n.pageContent,ID(n)
0,Lead Paint Law: Information for Owners and Sel...,<p>Maryland and federal laws about lead paint ...,100
1,Baltimore City Rent Escrow Law,<p>The purpose of the rent escrow law is to pr...,101
2,Cosigning a Lease,<p>There are times when a tenant needs a cosig...,111
3,When is Refusal-to-Rent Illegal Housing Discri...,<p>People often wrongly assume that any discri...,112
4,Anne Arundel County Rental and Housing Laws,\r\n <h2>Topics on this page:</h2>\r\n \r\n ...,749
...,...,...,...
452,Contesting a Will (Caveat Proceeding),<p>Contesting a will is a formal way of asking...,23350
453,Time limit for filing a will with the Register...,"\r\n \r\n <p>After someone dies, Maryland la...",23351
454,Youth Health Law,<h2>Topics on this page:</h2>\r\n \r\n <ul>\...,23352
455,Introduction to Juvenile Record Expungement,<p>Maryland law permits a person with a juveni...,23957


In [4]:
df.shape

(457, 3)

In [5]:
from bs4 import BeautifulSoup
import re
import string

#removing html code
def remove_html(text):
    soup = BeautifulSoup(text)
    text = soup.get_text()
    text = text.replace("\n", " ")
    text = text.replace("\xa0", " ")
    return text
df["n.pageContent"] = df["n.pageContent"].apply(remove_html)


In [29]:
import time

for i in range(0, 3):
  content = df["n.pageContent"][i]
  torch.cuda.empty_cache()
  big_content = content[:5000]

  # get the start time
  st = time.time()

  instruction = "Give me 3 questions based on the given content. Provide each questions in <ques> html tags.  "

  input_text = " The content is: " + big_content + " ." + instruction + " The 3 questions are:  "

  sequences = pipeline(
      input_text,
      max_length=2000,
      top_k=5,
      # num_return_sequences=1,
      # no_repeat_ngram_size = 5,
      # early_stopping = True,
      return_full_text=False
  )

  for seq in sequences:
      print(f"Result: {seq['generated_text']}")

  # get the end time
  et = time.time()

  # get the execution time
  elapsed_time = et - st
  print('Execution time:', elapsed_time, 'seconds')
  torch.cuda.empty_cache()



Result: 1. What are the duties of landlords and property owners of residential housing built before 1978 under Maryland’s lead paint law?  2. What are the required disclosures when selling a home built before 1978?  3. What are the lead-safe work practices that contractors must follow when renovating or repairing homes built before 1978?
Execution time: 5.338094711303711 seconds
Result: 1. What is the purpose of the rent escrow law in Baltimore City?  2. What conditions must exist for a tenant to use the rent escrow process?  3. What are the defenses available to a landlord in a rent escrow case?
Execution time: 3.461374521255493 seconds
Result: 1. What are the reasons why a tenant may need a cosigner?  2. What are the qualifications that a cosigner should have to be acceptable to a landlord?  3. What are the risks and benefits of cosigning a lease for a tenant?
Execution time: 3.631945848464966 seconds


In [8]:
content = df["n.pageContent"][0]
content

"Maryland and federal laws about lead paint are designed to reduce childhood lead poisoning. This article describes the duties of landlords and property owners of residential housing built before 1978. Information for renters and buyers can be found in another article.   Topics on this page:  Which homes are covered by Maryland’s lead paint law? Use of lead paint Owning and selling an Affected Property Landlord's duties Frequently asked questions for landlords    Which homes are covered by Maryland’s lead paint law? Only pre-1978 residential rental properties/units (“Affected Properties”) are regulated by Maryland’s Reduction of Lead Risk in Housing Act. Owner-occupied properties/units are not regulated. Affected Properties that are Maryland Department of the Environment (MDE) certified lead-free or limited lead-free are exempt from Maryland’s lead paint law. Affected Properties owned or operated by federal, State, or local government or by a public, quasi-public, or municipal corporat

In [9]:
import torch
torch.cuda.empty_cache()

In [10]:
big_content = content[:5000]
check_big_content_len = big_content.split(" ")
len(check_big_content_len)

784

In [11]:
big_content

"Maryland and federal laws about lead paint are designed to reduce childhood lead poisoning. This article describes the duties of landlords and property owners of residential housing built before 1978. Information for renters and buyers can be found in another article.   Topics on this page:  Which homes are covered by Maryland’s lead paint law? Use of lead paint Owning and selling an Affected Property Landlord's duties Frequently asked questions for landlords    Which homes are covered by Maryland’s lead paint law? Only pre-1978 residential rental properties/units (“Affected Properties”) are regulated by Maryland’s Reduction of Lead Risk in Housing Act. Owner-occupied properties/units are not regulated. Affected Properties that are Maryland Department of the Environment (MDE) certified lead-free or limited lead-free are exempt from Maryland’s lead paint law. Affected Properties owned or operated by federal, State, or local government or by a public, quasi-public, or municipal corporat

In [24]:
prompt=f"""
Follow exactly these 2 steps:
1. Read the user provided Document content below and aggregrate the data
Document content : """ + big_content + """
2. Answer the following question using only the above context.
User Question:  What is the responsibility of a cosigner if the tenant defaults on the lease?
If the user question is not related to the Document content,make sure to answer with "No related content found". Do not answer the question based on your own knowledge.
Answer:
"""

In [25]:
# get the start time
st = time.time()

sequences = pipeline(
    prompt,
    max_length=2000,
    top_k=5, #If you set k to 1, the model will only pick the top token
    early_stopping = True,
    # num_return_sequences=1,
    # no_repeat_ngram_size = 5,
    return_full_text=False,
    # temperature = 0
)

for seq in sequences:
    print(f"{seq['generated_text']}")

# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

torch.cuda.empty_cache()

The Document content does not mention anything about a cosigner or a defaulting tenant. Therefore, there is no related content found for the user question.
Execution time: 2.1337602138519287 seconds


In [ ]:
# get the start time
st = time.time()

instruction = "Give me 3 questions based on the given content. Provide each questions in <ques> html tags.  "

input_text = " The content is: " + big_content + " ." + instruction + " The 3 questions are:  "

sequences = pipeline(
    input_text,
    max_length=2000,
    top_k=5,
    # num_return_sequences=1,
    # no_repeat_ngram_size = 5,
    # early_stopping = True,
    return_full_text=False
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")

# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')
torch.cuda.empty_cache()

Result: 1. What is the purpose of the rent escrow law in Baltimore City?  2. What are the conditions that must exist for a tenant to use the rent escrow process?  3. What are the defenses that a landlord may have in a rent escrow case?
Execution time: 3.9178905487060547 seconds
